# Creazione dataset test per complicanze

In [1]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score, recall_score
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

excel_path = "Excel/Bilanciamento/RandomForest_Results_Bilanced_complicanze111.xlsx"

# Funzione per metriche
def get_metrics(y_true, y_predicted):
    matrix = confusion_matrix(y_true, y_predicted)
    accuracy = accuracy_score(y_true, y_predicted)
    precision = precision_score(y_true, y_predicted, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_predicted, average='weighted', zero_division=0)
    f1score = f1_score(y_true, y_predicted, average='weighted', zero_division=0)
    return matrix, accuracy, precision, recall, f1score

In [2]:
# 1) Vengono letti i due file
parte1 = pd.read_csv("Excel/Bilanciamento/parte1_bilanced.csv")
parte2 = pd.read_csv("Excel/Bilanciamento/parte2_v2_bilanced.csv")

# 1.1) Viene rinominata la feature ID in Patiene_ID per perettere il merge
parte2 = parte2.rename(columns={"ID": "Patient_ID"})

# 1.2) Viene fatto un inner‐merge su Patient_ID:
clust = parte1.merge(parte2, on="Patient_ID", how="inner")
clust = clust.fillna(clust.mean(numeric_only=True))

# 1.3) Vangono riempiti i valori null del dataset
clust = clust.fillna(clust.median(numeric_only=True))

# 2) Viene letto Diagnostics.csv per ricavare i codici associati a ciascun paziente
diag = pd.read_csv("Excel/Diagnostics.csv")

# 2.1) Per ogni paziente, viene prenso il Code con il maggior numero di occorrenze
diag_unique = (
    diag
    .groupby("Patient_ID")["Code"]
    .agg(lambda codes: codes.value_counts().idxmax())
    .reset_index()
    .rename(columns={"Code": "CodeStrat"})
)

# 2.2) Se hai codici troppo rari vengono raggruppati sotto "RARE" per evitare errori di stratify
code_counts = diag_unique["CodeStrat"].value_counts()
rare_codes = set(code_counts[code_counts < 2].index)
diag_unique["CodeStrat"] = diag_unique["CodeStrat"].apply(
    lambda c: "RARE" if c in rare_codes else c
)

# 3) Suddividi i soli pazienti “diagnosticati” 80/20, stratificando su CodeStrat
diag_train_patients, diag_test_patients = train_test_split(
    diag_unique["Patient_ID"],
    test_size=0.20,
    random_state=42,
    stratify=diag_unique["CodeStrat"]
)

# 4) Trova i pazienti in clust che NON compaiono in diag_unique (ossia “senza diagnosi”)
all_clust_patients = set(clust["Patient_ID"].unique())
diag_patients = set(diag_unique["Patient_ID"].unique())
no_diag_patients = list(all_clust_patients - diag_patients)
print(f"Numero pazienti senza diagnosi: {len(no_diag_patients)}")

# 5) Dividi i pazienti “senza diagnosi” (%) 80/20 in modo casuale
no_diag_train, no_diag_test = train_test_split(
    no_diag_patients,
    test_size=0.20,
    random_state=42
)

# 6) Costruisci la lista finale di test (diagnosticati nel 20% + senza diagnosi nel 20%)
test_patients  = set(diag_test_patients)  | set(no_diag_test)
train_patients = set(diag_train_patients) | set(no_diag_train)

# Verifica che non ci siano sovrapposizioni
assert train_patients.isdisjoint(test_patients), "Errore: un paziente è in entrambi i set!"

# 7) Filtra clust in base a Patient_ID: 
#    - se Patient_ID ∈ test_patients → va in df_test
#    - altrimenti va in df_train
df_train = clust[~clust["Patient_ID"].isin(test_patients)].copy()
df_test  = clust[ clust["Patient_ID"].isin(test_patients)].copy()

# 8) Rimuovi di nuovo la colonna Patient_ID, come richiesto, prima di salvare
#df_train = df_train.drop(columns=["Patient_ID", "Complicanze"])
#df_test = df_test.drop(columns=["Complicanze"])

# 9) Salva i due file risultanti
df_train.to_csv("Excel/Bilanciamento/Classification_train_bilanced.csv", index=False)
df_test.to_csv("Excel/Bilanciamento/Classification_test_bilanced.csv",  index=False)

print("Suddivisione completata:")
print(f"  - Train (Classification_train.csv): {len(df_train)} righe")
print(f"  - Test  (Classification_test.csv):  {len(df_test)}  righe")

NameError: name 'pd' is not defined

## Greedy Search

In [ ]:
from sklearn.model_selection import StratifiedKFold

train = pd.read_csv("Excel/Bilanciamento/Classification_train_bilanced.csv")
test = pd.read_csv("Excel/Bilanciamento/Classification_test_bilanced.csv")

test_patients1 = test['Patient_ID']
test = test.drop(columns=['Patient_ID'])
train = train.drop(columns=['Patient_ID'])

y_train = train['Has_Diagnostics']
y_test = test['Has_Diagnostics']
X_train = train.drop(columns=['Has_Diagnostics'])
X_test = test.drop(columns=['Has_Diagnostics'])

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Definisci lo spazio di ricerca
n_estimators_list = [50, 100, 150, 200, 250, 300, 350, 400]
max_depth_list = [5, 10, 15, 20]

best_config = None
best_score = 0  

prova_counter = 1

for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        print(f"▶️ Prova {prova_counter}: n_estimators={n_estimators}, max_depth={max_depth}")

        Prova = f"{prova_counter} - Est:{n_estimators}_Depth:{max_depth}"
        
        val_results = []

        for i, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), start=1):
            X_fold_train = X_train.iloc[train_idx]
            y_fold_train = y_train.iloc[train_idx]

            clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
            clf.fit(X_fold_train, y_fold_train)

            y_pred = clf.predict(X_test)

            cm, acc, prec, rec, f1 = get_metrics(y_test, y_pred)
            importances = clf.feature_importances_
            importances_dict = {f'Imp_{feat}': imp for feat, imp in zip(X_train.columns, importances)}

            val_results.append({
                'Prova': Prova,
                'KFold': i,
                'n_estimators': n_estimators,
                'max_depth': max_depth,
                'Accuracy': acc,
                'Precision': prec,
                'Recall': rec,
                'F1-Score': f1,
                'Confusion_Matrix': cm.tolist(),
                ' ': None,
                **importances_dict
            })

        df_nested = pd.DataFrame(val_results)

        # Calcola media F1
        mean_f1 = df_nested['F1-Score'].mean()

        # Aggiorna il best
        if mean_f1 > best_score:
            best_score = mean_f1
            best_config = (n_estimators, max_depth)
            print(f"✅ Nuova configurazione migliore trovata! F1 medio = {mean_f1:.4f}")

        # Salva su Excel
        if os.path.exists(excel_path):
            with pd.ExcelFile(excel_path) as reader:
                if 'Final1' in reader.sheet_names:
                    prev_nested = pd.read_excel(reader, sheet_name='Final1')
                    df_nested = pd.concat([prev_nested, df_nested], ignore_index=True)

        with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            df_nested.to_excel(writer, sheet_name='Final1', index=False)

        print(f"📁 Risultati prova {prova_counter} salvati.")
        prova_counter += 1

print(f"\n🏁 Greedy search terminata.")
print(f"⭐ Migliore configurazione: n_estimators={best_config[0]}, max_depth={best_config[1]}, con F1 medio = {best_score:.4f}")

In [ ]:
print("\n🔎 Analisi degli errori sulla configurazione migliore...")

# Ricrea il classificatore con i parametri migliori
n_estimators_best, max_depth_best = best_config
clf_best = RandomForestClassifier(n_estimators=n_estimators_best, max_depth=max_depth_best, random_state=42)
clf_best.fit(X_train, y_train)  # Allena sul full train

# Predizione su X_test
y_pred_best = clf_best.predict(X_test)

# Crea DataFrame con gli errori
df_resoults = pd.DataFrame({
    'Patient_ID': test_patients1,
    'True_Label': y_test,
    'Predicted_Label': y_pred_best
})
df_resoults['Errore'] = df_resoults['True_Label'] != df_resoults['Predicted_Label']

# Filtra solo gli errori
df_errors_only = df_resoults[df_resoults['Errore'] == True].copy()

diagnosi = pd.read_csv("Excel/Diagnostics.csv")

df_errors_only = df_errors_only.merge(diagnosi, on="Patient_ID", how="left")

df_errors_only = df_errors_only.drop(columns=['True_Label','Predicted_Label','Errore'])

df_errors_only = df_errors_only.sort_values(['Code'])

print(f"❌ Trovati {len(df_errors_only)} errori su {len(df_resoults)} pazienti nel test set.")

# Salva su Excel
with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    df_errors_only.to_excel(writer, sheet_name='Error_Analysis1', index=False)

print("Errori salvati sul foglio 'Error_Analysis'")

In [ ]:
diagnosi_filtrato = diagnosi[diagnosi["Code"].str.startswith("V")]
diagnosi_filtrato